In [ ]:
#@title String fields

path_to_image = './PATH_TO_IMAGE' #@param {type:"string"}
api_key = 'YOUR_API_KEY' #@param {type:"string"}
model = 'YOUR_MODEL_NAME' #@param {type:"string"}
version = 'YOUR_VERSION_NUMBER' #@param {type:"string"}

In [ ]:
!pip3 install requests_toolbelt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 296 kB/s 


In [ ]:
import numpy, io
from PIL import Image, ImageDraw
import cv2
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

## Helper Functions

In [ ]:
def infer_with_segmentation(image_path, api_key, model, version):
    # Load Image with PIL
    open_img = cv2.imread(image_path)
    color_img = cv2.cvtColor(open_img, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(color_img)

    # Convert to JPEG Buffer
    buffered = io.BytesIO()
    pil_image.save(buffered, quality=100, format="JPEG")

    # Build multipart form and post request
    m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})

    response = requests.post(f"https://outline.roboflow.com/{model}/{version}?api_key={api_key}", data=m, headers={'Content-Type': m.content_type})

    # formatted = json.dumps(response.json(), indent=2)
    # print(formatted)

    return response.json()

## Main Code

In [ ]:
# perform inference on the image
results = infer_with_segmentation(path_to_image, api_key, model, version)

# convert image to numpy (for convenience)
imArray = numpy.asarray(Image.open(path_to_image).convert("RGBA"))

# process each resulting prediction 
for index, prediction in enumerate(results["predictions"]):

  # convert polygon prediction points to workable tuple format
  polygon = [(point_obj["x"], point_obj["y"]) for point_obj in prediction["points"]]

  # create a mask image
  maskIm = Image.new('L', (imArray.shape[1], imArray.shape[0]), 0)
  ImageDraw.Draw(maskIm).polygon(polygon, outline=1, fill=1)

  # convert to numpy array
  maskArray = numpy.array(maskIm)

  # assemble new image (uint8: 0-255)
  newImArray = numpy.empty(imArray.shape,dtype='uint8')

  # colors (three first columns, RGB)
  newImArray[:,:,:3] = imArray[:,:,:3]

  # transparency (4th column)
  newImArray[:,:,3] = maskArray*255

  # convert back to Image from numpy and save
  newIm = Image.fromarray(newImArray, "RGBA")
  newIm.save(f"mask_{index}.png")